# Maple Wz Extractor

In [1]:
import sys
sys.path.append("src")

import src.main as main

wz_file = main.main(path="/Users/rush/projects/WzParser/assets/UI.wz", logging_path="./logs", build_tree=True)


/Users/rush/projects/WzParser/src/wzio/binaryreader.py:162: RuntimeWarning: overflow encountered in scalar multiply
  offset *= self.hash


Fetched 33037 nodes
Done parsing UI.wz


#### Data Structure debugging

In [5]:
# Json tree ???
with open("UI.json", "w") as f:
    f.write(wz_file.tree.to_json())

In [45]:
# If you do not get a tree view, it is because of a bug in the original source code
#https://stackoverflow.com/questions/46345677/treelib-prints-garbage-instead-of-pseudographics-in-python3
wz_file.tree.show(data_property="name")

UI.wz
├── Basic.img
│   ├── BtCancel
│   │   ├── disabled
│   │   │   └── 0
│   │   │       └── origin
│   │   ├── keyFocused
│   │   │   └── 0
│   │   │       └── origin
│   │   ├── mouseOver
│   │   │   └── 0
│   │   │       └── origin
│   │   ├── normal
│   │   │   └── 0
│   │   │       └── origin
│   │   └── pressed
│   │       └── 0
│   │           └── origin
│   ├── BtCancel2
│   │   ├── disabled
│   │   │   └── 0
│   │   │       └── origin
│   │   ├── keyFocused
│   │   │   └── 0
│   │   │       └── origin
│   │   ├── mouseOver
│   │   │   └── 0
│   │   │       └── origin
│   │   ├── normal
│   │   │   └── 0
│   │   │       └── origin
│   │   └── pressed
│   │       └── 0
│   │           └── origin
│   ├── BtCancel3
│   │   ├── disabled
│   │   │   └── 0
│   │   │       └── origin
│   │   ├── keyFocused
│   │   │   └── 0
│   │   │       └── origin
│   │   ├── mouseOver
│   │   │   └── 0
│   │   │       └── origin
│   │   ├── normal
│   │   │   └── 0
│   │   │       └── origin
│ 

In [2]:
def get_sub_tree_by_nodepath(tree, node_path):
    node = get_node_by_nodepath(node_path)
    if node is None:
        raise Exception(f"Node {node_path} not found")
    return tree.subtree(node.identifier)

def get_node_by_nodepath(node_path):
    try:
        return list(wz_file.tree.filter_nodes(lambda node: node.data.node_path == node_path))[0]
    except IndexError:
        return None

sub_tree = get_sub_tree_by_nodepath(wz_file.tree, "UI.wz/Logo.img/Nexon")
sub_tree.show(data_property="property_type")

WzPropertyType.Property
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └── WzPropertyType.Int
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └── WzPropertyType.Int
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └── WzPropertyType.Int
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └── WzPropertyType.Int
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └── WzPropertyType.Int
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └── WzPropertyType.Int
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └── WzPropertyType.Int
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └── WzPropertyType.Int
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └── WzPropertyType.Int
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └── WzPropertyType.Int
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └── WzPropertyType.Int
├── WzPropertyType.Canvas
│   ├── WzPropertyType.Vector
│   └──

##### Crypto

A simple example of string decryption. The cipher should be able to decrypt any Wz string in v83.

In [50]:
with open('./imhex/cipher.bin', 'rb') as f:
    cipher = f.read()

length = 12
mask = 0xAA

plain_text = ""
# Encrypted string
data = bytes([
    0x7B, 0x70, 0xFA, 0x65, 0x82, 0x17, 0x2F, 0x72, 0xDA, 0xAC, 0xDC, 0xB3
])
for i in range(length):
    b = data[i]
    b = b ^ (mask + i)
    b = b ^ cipher[i]
    plain_text += chr(b)

plain_text

'GuildBBS.img'

In [15]:
from src.crypto import WzKey

key =  WzKey()
key.at(512)

with open('./hexEditor/cipher.bin', 'wb') as f:
    f.write(key.cipher)

In [51]:
import binascii

val = binascii.hexlify(bytearray(cipher)).decode("utf-8")

chars = []
byte = "0x"
for i, char in enumerate(val):
    if i % 2 == 0 and i != 0:
        chars.append(byte)
        byte = "0x"
    byte += char.upper()

for i, char in enumerate(chars):
    if i % 16 == 0:
        print()
    print(char, end=", ")


0x96, 0xAE, 0x3F, 0xA4, 0x48, 0xFA, 0xDD, 0x90, 0x46, 0x76, 0x05, 0x61, 0x97, 0xCE, 0x78, 0x68, 
0x2B, 0xA0, 0x44, 0x8F, 0xC1, 0x56, 0x7E, 0x32, 0xFC, 0xE1, 0xF5, 0xB3, 0x14, 0x14, 0xC5, 0x22, 
0xF5, 0xC3, 0x68, 0x2E, 0x9D, 0xC3, 0x4A, 0x0B, 0xFA, 0xFE, 0x68, 0x45, 0x53, 0x8A, 0xFB, 0x5D, 
0x09, 0x4F, 0x59, 0xFC, 0xE9, 0x11, 0x12, 0x9B, 0xD9, 0x0F, 0xF2, 0xE8, 0x62, 0x69, 0x3B, 0x76, 
0x47, 0x88, 0x10, 0x75, 0xAC, 0xE3, 0x96, 0xD7, 0xDB, 0x12, 0x79, 0xCD, 0x59, 0xE4, 0xE0, 0x0C, 
0xEC, 0xA1, 0xFC, 0xD2, 0xB2, 0x3C, 0xBC, 0x4A, 0x5B, 0x9D, 0x6B, 0xA9, 0x86, 0x29, 0x28, 0x82, 
0x0B, 0x43, 0x59, 0x40, 0xDA, 0xE7, 0xE5, 0x93, 0x95, 0xDF, 0x00, 0x02, 0xEB, 0x8A, 0xBE, 0x0C, 
0xF1, 0x10, 0xAA, 0x5B, 0x67, 0x94, 0x90, 0xF4, 0xF5, 0xCE, 0xE8, 0xE1, 0x61, 0x09, 0x11, 0x09, 
0x50, 0xE2, 0x54, 0x4C, 0x05, 0x49, 0x97, 0x88, 0xF8, 0xF0, 0xAF, 0x15, 0x4A, 0x6C, 0xE0, 0x0E, 
0x5E, 0xC0, 0xCD, 0xAB, 0x52, 0x20, 0xA8, 0x2E, 0x50, 0x64, 0x0A, 0x1A, 0x00, 0x3E, 0x9E, 0xE9, 
0x6A, 0xD0, 0x19, 0xB5, 0xE1,